In [ ]:
'''
1) Берем данные, разделяем на тест и тренировку ( тренировку берем макс 10 процентов, больше нас не интересует)
2) Создаем модель нейронной сети
Функция
1) Берем обучающую выборку, выбираем первые три понедельника в качестве предикторов и один понедельник в качестве отклика
2) Подаем в функцию обучения, получаем результат
3) Смещаем значения на 1 шаг 
4) Обратно к пункту 2

Пример:
Пускай mondays - суммарная нагрузка в понедельник (в реальности сумму не используем)
mondays = [1, 4, 6, 7, 8, 10, 7, 11, 13, 14] 
Выбираем из mondays три первых понедельника в качестве предикторов и 1 в качестве отклика
to_fit = [[1, 4, 6],[7]]
Подаем в обучающую функцию и получаем какой-то результат
model.fit(to_fit)
Делаем смещение с шагом один в оригинальной выборке
to_fit = [[4, 6, 7], [8]]
Опять подаем в функцию обучения
model.fit(to_fit)
и т.д. до конца выборки.

Улавливаем время, когда нас устроит ошибка. Можно остановить обучение при определенном значении, либо сделать счетчик и запустить обучение модели заново
до определенного момента.

Проверяем работу модели на тестовой выборке. Важно запомнить с самого начала обучить модель на всех значениях, чтобы понять устраивает ли такая функция обучени или все
придумывать заново.
'''


'\n1) Берем данные, разделяем на тест и тренировку ( тренировку берем макс 10 процентов, больше нас не интересует)\n2) Создаем модель нейронной сети\nФункция\n1) Берем обучающую выборку, выбираем первые три понедельника в качестве предикторов и один понедельник в качестве отклика\n2) Подаем в функцию обучения, получаем результат\n3) Смещаем значения на 1 шаг \n4) Обратно к пункту 2\n\nПример:\nПускай mondays - суммарная нагрузка в понедельник (в реальности сумму не используем)\nmondays = [1, 4, 6, 7, 8, 10, 7, 11, 13, 14] \nВыбираем из mondays три первых понедельника в качестве предикторов и 1 в качестве отклика\nto_fit = [[1, 4, 6],[7]]\nПодаем в обучающую функцию и получаем какой-то результат\nmodel.fit(to_fit)\nДелаем смещение с шагом один в оригинальной выборке\nto_fit = [[4, 6, 7], [8]]\nОпять подаем в функцию обучения\nmodel.fit(to_fit)\nи т.д. до конца выборки.\n\nУлавливаем время, когда нас устроит ошибка. Можно остановить обучение при определенном значении, либо сделать счетчи

In [ ]:
# Adding libraries 
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv, read_excel
import math
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM, GlobalMaxPool1D, Dropout, Flatten
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import TensorBoard 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [ ]:
# convert an array of values into a dataset matrix 
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], [] 
	for i in range((int((len(dataset) - 3 * look_back) / look_back))):
		j = i * look_back # next monday
		k = (i + 3) * look_back # step
		a = dataset[j : k, 0]
		dataX.append(a)
		b = dataset[k : k + look_back, 0]
		dataY.append(b)

	return numpy.array(dataX), numpy.array(dataY)
 
 

In [ ]:
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
# load the dataset
dataframe = read_csv('mondays_to_predict.csv', usecols=[1], engine='python')
dataset = dataframe.values
dataset = dataset.astype('float32')
print(len(dataset))

4992


In [ ]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [ ]:
# split into train and test sets

train_size = int(len(dataset)) - 96
test_size = int(len(dataset)) - 4 * 96
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(train.shape)
print(test.shape)
''' Надо изменить в функции разбивки массива на примеры и отклики длинну цикла,
чтобы это сделать нужно убрать последний понедельник и сделать его якобы тем, что мы желаем получить ( оставляем для сравнени с тем, что будет после ф-ии predict). 
Далее из того, что получилось режешь понедельники на примеры и отклики,
но теперь понимаем, что три последних понедельника выпадают. Их необходимо пихнуть в качестве теста того, что желаем получить, но при этом они остаются в выборке тоже, как ответы и части
примеров. То есть из получившегося N (4896) массива получается (N - 3 * 96) / 96 примеров и откликов. Это будет максимальное количество i в цикле. Итоговый конец цикла следующий
range(((len(dataset) - 3 * look_back) / look_back) - 1) вроде получается 48 итераций от 0 до 47. Далее это все дело превращаем в 2 трехмерных массива один с примерами, другой с откликами'''

(4896, 1)
(96, 1)


' В общем что тебе надо, так это изменить в функции разбивки массива на примеры и отклики длинну цикла,\nчтобы это сделать нужно убрать последний понедельник и сделать его якобы тем, что мы желаем получить ( оставляем для сравнени с тем, что будет после ф-ии predict). \nДалее из того, что получилось режешь понедельники на примеры и отклики,\nно теперь понимаем, что три последних понедельника выпадают. Их необходимо пихнуть в качестве теста того, что желаем получить, но при этом они остаются в выборке тоже, как ответы и части\nпримеров. То есть из получившегося N (4896) массива получается (N - 3 * 96) / 96 примеров и откликов. Это будет максимальное количество i в цикле. Итоговый конец цикла следующий\nrange(((len(dataset) - 3 * look_back) / look_back) - 1) вроде получается 48 итераций от 0 до 47. Далее это все дело превращаем в 2 трехмерных массива один с примерами, другой с откликами'

In [ ]:
# reshape
look_back = 96
trainX, trainY = create_dataset(train, look_back)
testX = dataset[test_size : train_size,:]
testX = numpy.array(testX)
testY = numpy.array(test)
#testX, testY = create_dataset(test, look_back)
#trainX = trainX.reshape(12, 32, 12)
#trainY = trainY.reshape(12, 32)
#testX = testX.reshape(12, 32)
trainX


array([[0.0000000e+00, 2.9098243e-05, 5.8195554e-05, ..., 4.1813366e-02,
        4.1842468e-02, 4.1871563e-02],
       [1.9553641e-02, 1.9582739e-02, 1.9611837e-02, ..., 6.1367013e-02,
        6.1396107e-02, 6.1425209e-02],
       [3.9107285e-02, 3.9136380e-02, 3.9165474e-02, ..., 8.0920652e-02,
        8.0949746e-02, 8.0978848e-02],
       ...,
       [8.7991387e-01, 8.7994295e-01, 8.7997204e-01, ..., 9.2172724e-01,
        9.2175633e-01, 9.2178541e-01],
       [8.9946753e-01, 8.9949661e-01, 8.9952570e-01, ..., 9.4128090e-01,
        9.4130999e-01, 9.4133908e-01],
       [9.1902113e-01, 9.1905028e-01, 9.1907936e-01, ..., 9.6083450e-01,
        9.6086365e-01, 9.6089274e-01]], dtype=float32)

In [ ]:
trainY

array([[0.05770387, 0.05790423, 0.05810459, ..., 0.0763374 , 0.07653777,
        0.07673813],
       [0.07693849, 0.07713885, 0.07733921, ..., 0.09557203, 0.09577239,
        0.09597275],
       [0.09617312, 0.09637348, 0.09657383, ..., 0.11480665, 0.11500701,
        0.11520737],
       ...,
       [0.9232619 , 0.9234622 , 0.9236626 , ..., 0.9418954 , 0.94209576,
        0.94229615],
       [0.9424965 , 0.94269687, 0.9428972 , ..., 0.96113   , 0.9613304 ,
        0.96153075],
       [0.96173114, 0.96193147, 0.96213186, ..., 0.9803647 , 0.980565  ,
        0.9807654 ]], dtype=float32)

In [ ]:
print(trainX.shape)
print(trainY.shape)

(48, 288)
(48, 96)


In [ ]:
print(testX.shape)
print(testY.shape)

(288, 1)
(96, 1)


In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 288, 1))
#trainX = numpy.reshape(trainX, (trainX.shape[0], 24, 12))
trainY = numpy.reshape(trainY,(trainY.shape[0], 96, 1))
#trainY = numpy.reshape(trainY,(trainY.shape[0], int(trainY.shape[1] / 12), 12))
testX = numpy.reshape(testX, (1, 288, 1))
#testX = numpy.reshape(testX, (1, int(testX.shape[0] / 12), 12))
testY = numpy.reshape(testY, (1, 96, 1))
#testY = numpy.reshape(testY, (1, int(testY.shape[0] / 12), 12))
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(48, 288, 1)
(48, 96, 1)
(1, 288, 1)
(1, 96, 1)


In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

(48, 288, 1)
(48, 96, 1)
(1, 288, 1)
(1, 96, 1)


In [ ]:
from keras.regularizers import l2

In [ ]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=1000)

In [ ]:
model = Sequential()
model.add(LSTM(48, activation='relu', batch_input_shape = (1, 288, 1), stateful = True, kernel_regularizer = l2(0.01), recurrent_regularizer = l2(0.01), bias_regularizer = l2(0.01) ))
#model.add(LSTM(48, activation='relu', batch_input_shape = (1, 24, 12), stateful = True))
model.add(RepeatVector(96))
model.add(Dropout(0.3))
model.add(LSTM(48, activation='relu', batch_input_shape = (1, 288, 1), stateful = True, return_sequences=True, kernel_regularizer = l2(0.01), recurrent_regularizer = l2(0.01), bias_regularizer = l2(0.01)))
#model.add(LSTM(48, activation='relu', batch_input_shape = (1, 24, 12), stateful = True, return_sequences=True))
model.add(Dropout(0.3))
#model.add(TimeDistributed(Dense(288)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse', metrics = ['mape'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (1, 48)                   9600      
_________________________________________________________________
repeat_vector_1 (RepeatVecto (1, 96, 48)               0         
_________________________________________________________________
dropout_1 (Dropout)          (1, 96, 48)               0         
_________________________________________________________________
lstm_2 (LSTM)                (1, 96, 48)               18624     
_________________________________________________________________
dropout_2 (Dropout)          (1, 96, 48)               0         
_________________________________________________________________
dense_1 (Dense)              (1, 96, 1)                49        
Total params: 28,273
Trainable params: 28,273
Non-trainable params: 0
__________________________________________________

In [ ]:
# fit network
tbCallBack = TensorBoard(log_dir='/content', histogram_freq=0, write_graph=True, write_images=True)
tensorboard=TensorBoard(log_dir='/content', write_graph=True)
#for i in range(100):
for i in range(trainX.shape[0]):
    X = trainX[i]
    X = numpy.reshape(X, (1, 288, 1))
    Y = trainY[i]
    Y = numpy.reshape(Y, (1, 96, 1))
    #X = numpy.reshape(X, (1, 24, 12))
    #Y = trainY[i]
    #Y = numpy.reshape(Y, (1, 8, 12))
    #model.fit(trainX, trainY, epochs=1, batch_size = 1,  verbose = 1, shuffle = False, callbacks=[es])
    model.fit(trainX, trainY, epochs=1, batch_size = 1,  verbose = 1, shuffle = False)
    model.reset_states()

Epoch 1/1
48/48 [==============================] - 4s 88ms/step - loss: 2.0544 - mape: 37.6322
Epoch 1/1
48/48 [==============================] - 3s 69ms/step - loss: 1.2670 - mape: 67.5768
Epoch 1/1
48/48 [==============================] - 3s 70ms/step - loss: 0.9296 - mape: 61.5212
Epoch 1/1
48/48 [==============================] - 3s 72ms/step - loss: 0.7684 - mape: 59.4035
Epoch 1/1
48/48 [==============================] - 3s 72ms/step - loss: 0.6685 - mape: 55.6838
Epoch 1/1
48/48 [==============================] - 3s 73ms/step - loss: 0.5991 - mape: 60.1672
Epoch 1/1
48/48 [==============================] - 4s 73ms/step - loss: 0.5348 - mape: 58.1564
Epoch 1/1
48/48 [==============================] - 4s 74ms/step - loss: 0.4836 - mape: 60.5944
Epoch 1/1
48/48 [==============================] - 3s 73ms/step - loss: 0.4338 - mape: 60.2654
Epoch 1/1
48/48 [==============================] - 4s 73ms/step - loss: 0.3894 - mape: 59.9928
Epoch 1/1
48/48 [==============================] -

In [ ]:
# re-define the batch size
n_batch = 1

In [ ]:
# re-define model
new_model = Sequential()
new_model.add(LSTM(10, batch_input_shape=(n_batch, 3, 1), stateful=True))
new_model.add(Dense(1))

In [ ]:
# copy weights
old_weights = model.get_weights()
new_model.set_weights(old_weights)

In [ ]:
# compile model
new_model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# make predictions
#trainPredict = model.predict(trainX, batch_size=1)
#model.reset_states()
testPredict = model.predict(testX, batch_size=1)
print(testPredict, testY)

[[[0.55464494]
  [0.73495615]
  [0.8402823 ]
  [0.9018159 ]
  [0.9377677 ]
  [0.95877385]
  [0.97104764]
  [0.9782192 ]
  [0.9824096 ]
  [0.984858  ]
  [0.98628855]
  [0.9871243 ]
  [0.9876128 ]
  [0.9878981 ]
  [0.9880649 ]
  [0.9881623 ]
  [0.9882192 ]
  [0.9882524 ]
  [0.98827195]
  [0.9882832 ]
  [0.98828983]
  [0.9882937 ]
  [0.988296  ]
  [0.98829734]
  [0.98829806]
  [0.98829854]
  [0.9882988 ]
  [0.98829895]
  [0.9882991 ]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829

In [ ]:
print(testPredict)

[[[0.55464494]
  [0.73495615]
  [0.8402823 ]
  [0.9018159 ]
  [0.9377677 ]
  [0.95877385]
  [0.97104764]
  [0.9782192 ]
  [0.9824096 ]
  [0.984858  ]
  [0.98628855]
  [0.9871243 ]
  [0.9876128 ]
  [0.9878981 ]
  [0.9880649 ]
  [0.9881623 ]
  [0.9882192 ]
  [0.9882524 ]
  [0.98827195]
  [0.9882832 ]
  [0.98828983]
  [0.9882937 ]
  [0.988296  ]
  [0.98829734]
  [0.98829806]
  [0.98829854]
  [0.9882988 ]
  [0.98829895]
  [0.9882991 ]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829913]
  [0.98829

In [ ]:
print(testY)

[[[0.9972357 ]
  [0.9972648 ]
  [0.9972939 ]
  [0.997323  ]
  [0.99735206]
  [0.99738115]
  [0.99741036]
  [0.99743944]
  [0.99746853]
  [0.9974976 ]
  [0.9975267 ]
  [0.9975558 ]
  [0.9975849 ]
  [0.99761397]
  [0.99764305]
  [0.99767214]
  [0.9977012 ]
  [0.99773043]
  [0.9977595 ]
  [0.9977886 ]
  [0.9978177 ]
  [0.9978468 ]
  [0.99787587]
  [0.99790496]
  [0.99793404]
  [0.99796313]
  [0.9979922 ]
  [0.9980213 ]
  [0.9980505 ]
  [0.9980796 ]
  [0.9981087 ]
  [0.9981378 ]
  [0.99816686]
  [0.99819595]
  [0.99822503]
  [0.9982541 ]
  [0.9982832 ]
  [0.9983123 ]
  [0.9983414 ]
  [0.9983706 ]
  [0.9983997 ]
  [0.99842876]
  [0.99845785]
  [0.99848694]
  [0.998516  ]
  [0.9985451 ]
  [0.9985742 ]
  [0.9986033 ]
  [0.9986324 ]
  [0.99866146]
  [0.99869066]
  [0.99871975]
  [0.99874884]
  [0.9987779 ]
  [0.998807  ]
  [0.9988361 ]
  [0.9988652 ]
  [0.9988943 ]
  [0.99892336]
  [0.99895245]
  [0.99898154]
  [0.9990106 ]
  [0.9990398 ]
  [0.9990689 ]
  [0.999098  ]
  [0.9991271 ]
  [0.99915

In [ ]:
score = model.evaluate(testX, testY, verbose = 1, batch_size= 1 )
print('Test loss:', score[0])
print('Test Mean Procentage Error:', score[1])

1/1 [==============================] - 0s 334ms/step
Test loss: 0.025874149054288864
Test Mean Procentage Error: 1.5003854036331177


In [ ]:
numpy.reshape(testY, (1, 96))
#numpy.reshape(testY, (8, 12))
testY

In [ ]:
testPredict = numpy.reshape(testPredict, (96))
testY = numpy.reshape(testY,(96))

In [ ]:
type(testPredict)

numpy.ndarray

In [ ]:
numpy.savetxt('testPredict_new.csv', testPredict, fmt='%1.8f')
numpy.savetxt('testY_new.csv', testY, fmt='%1.8f')

In [ ]:
# invert predictions
#trainPredict = scaler.inverse_transform(trainPredict)
#trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# calculate root mean squared error
#trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
#print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# shift train predictions for plotting
#trainPredictPlot = numpy.empty_like(dataset)
#trainPredictPlot[:, :] = numpy.nan
#trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
#print(trainPredict)

In [ ]:
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
print(testPredict)
print(testPredict.shape)

In [ ]:
# plot baseline and predictions
fig,ax = plt.subplots(figsize=(100,100))
ax.plot(scaler.inverse_transform(dataset), color = 'k')
ax.plot(trainPredictPlot,color = 'g')
ax.plot(testPredictPlot, color = 'y')
plt.savefig('2LSTMdropout.png')
plt.show()